In [2]:
import pandas as pd
import requests
import string
import time
import config
import datadotworld as dw
import os
import pygsheets
import json
from io import StringIO
from pandas.io.json import json_normalize

In [27]:
#URL and session variables
url_base = 'https://mainecampaignfinance.com'

s = requests.Session()
cookies = requests.cookies.RequestsCookieJar()

#POST headers
headers = {'Host':'mainecampaignfinance.com'
            ,'Origin': url_base
            ,'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:69.0) Gecko/20100101 Firefox/69.0'
            ,'Accept': 'application/octet-stream'
            ,'Accept-Language': 'en-US,en;q=0.5'
            ,'Accept-Encoding': 'gzip, deflate, br'
            ,'Referer':'https://mainecampaignfinance.com/'
            ,'Content-Type': 'application/json;charset=utf-8'
            ,'Connection': 'keep-alive'
            ,'TE':'Trailers'
            ,'Pragma': 'no-cache'
            ,'Cache-Control': 'no-cache'}

#Parameters
year = '2020,2019'

In [50]:
def get_cands(year,headers):

    #GET CANDIDATES LOOKUP#
    url='https://mainecampaignfinance.com/api///Organization/SearchCandidates'

    data = {"ElectionYear":year,
     "pageNumber":1,
     "pageSize":2147483647}

    r = s.post(url, data=json.dumps(data), headers=headers).json()

    df = json_normalize(r)
    
    length = len(df)

    with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance','candidate_lookup.csv') as w:
        df.to_csv(w,index=False)
        
    return length

def get_trans(year,headers):

    url = 'https://mainecampaignfinance.com/api///Search/TransactionSearchInformationExpExportToCSV'

    #Parameters for looping through search
    committee_types = {'candidate':'01'
                      ,'bqc':'02'
                      ,'pac':'03'
                      ,'party-committee':'09'}

    transaction_types = {'contributions':'CON'
                        ,'expenditures':'EXP'
                        ,'independent_expenditures':'IE'
                        ,'ballot_questions':'All'}

    trans_types = list(transaction_types.values())
    filenames = list(transaction_types.keys())
    comm_types = list(committee_types.values())

    #Data dictionary for query
    data = {"ElectionYear":year
            ,"pageNumber":'1'
            ,"pageSize":'2147483647' #Sets max responses from page (defaults to 10)
            ,"ValidationRequired":'0'}


    for idx, b in enumerate(transaction_types):

        data.update({"TransactionType":trans_types[idx]})

        dfs=[]

        #Reset index

        for i, n in enumerate(comm_types):

            data.update({"CommitteeType":comm_types[i]})

            r = s.post(url, data=json.dumps(data), headers=headers)
            df = pd.read_csv(StringIO(r.content.decode('utf-8')))
            dfs.append(df)
        

        #COMBINE dfs
        df=pd.concat(dfs,sort=False,ignore_index=True).drop_duplicates()

        length = len(df)
        
        file = filenames[idx]

#         count = dw.query('darrenfishell/2020-maine-state-campaign-finance', 'SELECT COUNT(*) FROM '+filenames[idx]).dataframe['count'][0]
#         if count < len(df):
        with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance',file+'.csv') as w:
            df.to_csv(w,index=False)
        print(f'Wrote file {file}.csv to data.world, at {length} records.')
        
def write_to_gsheet():
    
    #ddw query
    a65bf908-26ba-4f11-b413-a57bd8b3a9f5

In [51]:
# Execute queries
try:
    length = get_cands(year,headers)
    print(f'Wrote records for {length} candidates.')
except:
    print('Failed to write candidate lookup.')

try: 
    get_trans(year,headers)
    print('Contributions loaded')
except:
    print('Failed to write transactions')

0
0
0
0
Wrote file contributions.csv to data.world, at 22037 records.
1
1
1
1
Wrote file expenditures.csv to data.world, at 8150 records.
2
2
2
2
Wrote file independent_expenditures.csv to data.world, at 24 records.
3
3
3
3
Wrote file ballot_questions.csv to data.world, at 0 records.
Contributions loaded


In [ ]:
##QUESTION 1 COMMITTEES##

# #GET QUESTION ONE COMMITTEES#
# url='https://mainecampaignfinance.com/api///Organization/SearchAssociatedBallotQuestionCommittees'

# vax_dict = {"BallotQuestionTypeId":"322"
#             ,"ElectionYear":"2020"
#             ,"pageNumber":"1"
#             ,"pageSize":'10'
#             ,"sortDir":"desc"
#             ,"sortedBy":""}

# r = s.post(url, data=json.dumps(vax_dict), headers=headers).json()

# df = json_normalize(r)

# with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance','question-one-committees.csv') as w:
#     df.to_csv(w,index=False)